In [613]:
import pandas as pd
import numpy as np

In [614]:
df = pd.read_excel('../dataset_final/main_df.xlsx', index_col=0)

In [615]:
df_work = pd.read_excel('../dataset_final/비재무최종_df.xlsx', index_col=0)

In [616]:
# 평균근속연수(년) update
df['평균근속연수(년)'] = df_work['평균근속연수']

In [617]:
df_k2 = pd.read_excel('../dataset_real_final/K2_score_real_final_2.xlsx', index_col=0)
df_k2

,회사명,거래소코드,회계년도,log총자산(원),총자산회전율(log),누적수익성,재무구조,K2_score,K2_부실판단
0,(주)CMG제약,58820,2011/12,10.436245,-0.187005,-1.655914,4.072802,-31.590655,부실가능
1,(주)CMG제약,58820,2012/12,10.625381,-0.414383,-1.156158,5.900594,-19.952228,부실가능
2,(주)CMG제약,58820,2013/12,10.752128,-0.442452,-0.925333,3.727510,-17.767189,부실가능
3,(주)CMG제약,58820,2014/12,10.755802,-0.398771,-0.917659,8.975031,-11.478181,부실가능
4,(주)CMG제약,58820,2015/12,10.758159,-0.401128,-0.949445,19.555639,0.016272,판정보류
...,...,...,...,...,...,...,...,...,...
17482,흥구석유(주),24060,2018/12,10.889631,0.331022,0.434820,4.900155,12.752695,건전판정
17483,흥구석유(주),24060,2019/12,10.900004,0.278019,0.470459,13.670314,23.379678,건전판정
17484,흥구석유(주),24060,2020/12,10.922681,0.137426,0.501236,19.906224,30.787784,건전판정
17485,흥구석유(주),24060,2021/12,10.920041,0.201236,0.515682,27.143226,39.535530,건전판정


In [618]:
# K2_score update
df['K2_score'] = df_k2['K2_score']
df['K2_부실판단'] = df_k2['K2_부실판단']

In [619]:
df_share_max = pd.read_excel('../dataset_final/대주주주식보유비율_최종.xlsx', index_col=0)
df_share_max

,회사명,회계년도,거래소코드,보통주식비율
0,(주)CMG제약,2011/12,58820,2.31
1,(주)CMG제약,2012/12,58820,30.29
2,(주)CMG제약,2013/12,58820,26.80
3,(주)CMG제약,2014/12,58820,26.80
4,(주)CMG제약,2015/12,58820,26.29
...,...,...,...,...
17482,흥구석유(주),2018/12,24060,37.45
17483,흥구석유(주),2019/12,24060,35.56
17484,흥구석유(주),2020/12,24060,34.23
17485,흥구석유(주),2021/12,24060,34.23


In [620]:
# 최대주주지분율 Update
df['최대주주특수관계인_지분율(%)'] = df_share_max['보통주식비율']

---

# I. 공통   

In [621]:
df.head(1)

,회사명,거래소코드,회계년도,주거래은행,평균근속연수(년),R&D집중도(%),업력(년),최대주주특수관계인_지분율(%),최대주주변경횟수(회),감사법인변경횟수(회),...,PPI(2015기준),실질GDP성장률(%),외화포지션(%),K2_score,K2_부실판단,EV/EBITDA(배),52주베타(배),PER,PBR,PCR
0,(주)CMG제약,58820,2011,KEB하나은행,3.666667,3.32,10,2.31,0,0,...,105.71,3.7,154.163051,-31.590655,부실가능,18.37,0.070097,0.0,2.21,0.0


## I-1. Target'삭제'기준 Rows -> Drop
    - rows : 17,487 -> 11,248 

In [622]:
main_df = df.loc[~(df['Target']=='삭제')]
main_df.shape

(11248, 62)

## I-2. 컬럼별 info() 확인

In [623]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11248 entries, 0 to 17486
Data columns (total 62 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   회사명                  11248 non-null  object 
 1   거래소코드                11248 non-null  int64  
 2   회계년도                 11248 non-null  int64  
 3   주거래은행                11248 non-null  object 
 4   평균근속연수(년)            11081 non-null  float64
 5   R&D집중도(%)            11141 non-null  float64
 6   업력(년)                11248 non-null  int64  
 7   최대주주특수관계인_지분율(%)     11248 non-null  float64
 8   최대주주변경횟수(회)          11248 non-null  int64  
 9   감사법인변경횟수(회)          11248 non-null  int64  
 10  직원수증감율(%)            11110 non-null  float64
 11  수출기업여부(dummy)        11248 non-null  int64  
 12  기간제비율(%)             11168 non-null  float64
 13  표준산업분류_중분류           7162 non-null   object 
 14  산업위험_경기민감도           11248 non-null  object 
 15  Target               11248 non-null 

In [624]:
# 거래소코드 type 변경 : int -> str
main_df['거래소코드'] = main_df['거래소코드'].astype('str').str.zfill(6)
# main_df['거래소코드']

C:\Users\zino0\AppData\Local\Temp\ipykernel_31300\947282485.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['거래소코드'] = main_df['거래소코드'].astype('str').str.zfill(6)


## I-3. columns 조정
- 컬럼 순서 조정
    - 재무 -> 비재무 -> Target 순

In [625]:
main_df_copy = main_df.copy()

In [626]:
main_df = main_df.reindex(columns=[
       # 기본
       '회사명', '거래소코드', '회계년도',
       
       # 안정성
       '유동비율(%)', '당좌비율(%)', '비유동비율(%)',
       '부채비율(%)', '유동부채비율(%)', '차입금의존도(%)', 
       '이익잉여금비율(%)', '순운전자본비율(%)',
       '비유동장기적합률(%)', '이자보상배율(이자비용)(배)', '외화포지션(%)',

       # 수익성
       '총자본순이익률(%)', '자기자본순이익률(%)', '경영자본순이익률(%)', '매출액순이익률(%)',
       '총자본정상영업이익률(%)', '자기자본정상영업이익률(%)', '매출액정상영업이익률(%)', 
       '금융비용부담률(%)',  

       # 활동성
       '총자산회전율(배)','매출채권회전률(배)', '당좌자산회전률(배)',
       '재고자산회전률(배)', '유형자산회전율(배)', '매입채무회전률(배)', 
       '자기자본회전률(배)', '경영자본회전률(배)',
         
       # 성장성
       '총자산_증가율(%)','유동자산증가율(%)',
       '매출액증가율(%)', '정상영업이익증가율(%)', '순이익증가율(%)', 
             
       # 거시
       'spread(%)', 'PPI(2015기준)','실질GDP성장률(%)',

       # 시장정보_주가관련
       'EV/EBITDA(배)','52주베타(배)', 'PER', 'PBR', 'PCR',

       # 기타_재무
       'CASH FLOW 대 부채비율(%)', '총자본투자효율(%)',
       '평균총자산(천원)', '매출액(수익)(천원)',
       'K2_score', 'K2_부실판단',
       
       
       # 비재무정보_기업위험요인
       # 산업위험
       '산업위험_경기민감도','표준산업분류_중분류', 
       
       # 경영위험
       '최대주주특수관계인_지분율(%)', '최대주주변경횟수(회)', '감사법인변경횟수(회)', '업력(년)',

       # 영업위험
       '평균근속연수(년)', '직원수증감율(%)', '기간제비율(%)', 
       'R&D집중도(%)', '수출기업여부(dummy)',
        
       # 기타_비재무
       '주거래은행',

       # Target 
       'Target'])

In [627]:
# 총자산_증가율(%) 제거 - 결측치(2011') 많음 & 성장성 카테고리의 기타 변수 충분하다 판단 
main_df.drop('총자산_증가율(%)', axis=1, inplace=True)
# main_df.head(1)

### 조정 columns

In [628]:
# 기업규모 : 평균총자산 컬럼 log 처리 
main_df['평균총자산(천원)'] = np.log(main_df['평균총자산(천원)'])

In [629]:
# 외화포지션이름 변경 :외화포지션(%) -> 외화포지션(배) 

In [630]:
main_df.columns = [
       # 기본
       '회사명', '거래소코드', '회계년도',
       
       # 안정성
       '유동비율(%)', '당좌비율(%)', '비유동비율(%)',
       '부채비율(%)', '유동부채비율(%)', '차입금의존도(%)', 
       '이익잉여금비율(%)', '순운전자본비율(%)',
       '비유동장기적합률(%)', '이자보상배율(이자비용)(배)', '외화포지션(배)',

       # 수익성
       '총자본순이익률(%)', '자기자본순이익률(%)', '경영자본순이익률(%)', '매출액순이익률(%)',
       '총자본정상영업이익률(%)', '자기자본정상영업이익률(%)', '매출액정상영업이익률(%)', 
       '금융비용부담률(%)',  

       # 활동성
       '총자산회전율(배)','매출채권회전률(배)', '당좌자산회전률(배)',
       '재고자산회전률(배)', '유형자산회전율(배)', '매입채무회전률(배)', 
       '자기자본회전률(배)', '경영자본회전률(배)',
         
       # 성장성
       '유동자산증가율(%)', '매출액증가율(%)', '정상영업이익증가율(%)', '순이익증가율(%)', 
             
       # 거시
       'spread(%)', 'PPI(2015기준)','실질GDP성장률(%)',

       # 시장정보_주가관련
       'EV/EBITDA(배)','52주베타(배)', 'PER', 'PBR', 'PCR',

       # 기타_재무
       'CASH FLOW 대 부채비율(%)', '총자본투자효율(%)',
       'log_평균총자산', '매출액(수익)(천원)',
       'K2_score', 'K2_부실판단',
       
       
       # 비재무정보_기업위험요인
       # 산업위험
       '산업위험_경기민감도','표준산업분류_중분류', 
       
       # 경영위험
       '최대주주특수관계인_지분율(%)', '최대주주변경횟수(회)', '감사법인변경횟수(회)', '업력(년)',

       # 영업위험
       '평균근속연수(년)', '직원수증감율(%)', '기간제비율(%)', 
       'R&D집중도(%)', '수출기업여부(dummy)',
        
       # 기타_비재무
       '주거래은행',

       # Target 
       'Target']

In [631]:
main_df_copy_2 = main_df.copy()

In [632]:
main_df.drop('표준산업분류_중분류', axis= 1, inplace= True)

In [633]:
# 처리 전 main_df

# df.to_excel('../dataset_real_final/main_df_raw.xlsx')


# II. 결측치

In [634]:
# 컬럼별 결측치 Check
for i in main_df.columns : 
    print(f'{i} : {main_df[i].isna().sum()}')
    print('________________________')
    
main_df.shape

회사명 : 0
________________________
거래소코드 : 0
________________________
회계년도 : 0
________________________
유동비율(%) : 96
________________________
당좌비율(%) : 96
________________________
비유동비율(%) : 96
________________________
부채비율(%) : 96
________________________
유동부채비율(%) : 96
________________________
차입금의존도(%) : 96
________________________
이익잉여금비율(%) : 192
________________________
순운전자본비율(%) : 96
________________________
비유동장기적합률(%) : 96
________________________
이자보상배율(이자비용)(배) : 96
________________________
외화포지션(배) : 192
________________________
총자본순이익률(%) : 96
________________________
자기자본순이익률(%) : 96
________________________
경영자본순이익률(%) : 96
________________________
매출액순이익률(%) : 96
________________________
총자본정상영업이익률(%) : 96
________________________
자기자본정상영업이익률(%) : 96
________________________
매출액정상영업이익률(%) : 96
________________________
금융비용부담률(%) : 96
________________________
총자산회전율(배) : 192
________________________
매출채권회전률(배) : 96
________________________
당좌자산회전률(배) : 96
________________

(11248, 60)

## II-1. 재무비율 공통 NaN 제거
    - 96rows 제거 
    - 11,248 -> 11,152
    - 'Target' = 1 : 8 rows 제거

In [635]:
# 재무정보 columns : 공통적으로 결측치 존재 확인
main_df.loc[main_df['당좌자산회전률(배)'].isna()][['유동비율(%)', '당좌비율(%)', '유동부채비율(%)', '비유동비율(%)', '순운전자본비율(%)', '부채비율(%)',
       '차입금의존도(%)', '비유동장기적합률(%)', '자기자본순이익률(%)', '총자본순이익률(%)',
       '자기자본정상영업이익률(%)', '총자본정상영업이익률(%)', '경영자본순이익률(%)', '매출액순이익률(%)',
       '이자보상배율(이자비용)(배)', '금융비용부담률(%)', '매출액정상영업이익률(%)', '당좌자산회전률(배)',
       '재고자산회전률(배)', '자기자본회전률(배)', '경영자본회전률(배)', '매출채권회전률(배)', '매입채무회전률(배)',
       '유형자산회전율(배)', '매출액증가율(%)', '순이익증가율(%)', '정상영업이익증가율(%)', '유동자산증가율(%)',
       'log_평균총자산', '매출액(수익)(천원)', '이익잉여금비율(%)', '총자산회전율(배)',
       'CASH FLOW 대 부채비율(%)']]#.isna().sum()

,유동비율(%),당좌비율(%),유동부채비율(%),비유동비율(%),순운전자본비율(%),부채비율(%),차입금의존도(%),비유동장기적합률(%),자기자본순이익률(%),총자본순이익률(%),...,유형자산회전율(배),매출액증가율(%),순이익증가율(%),정상영업이익증가율(%),유동자산증가율(%),log_평균총자산,매출액(수익)(천원),이익잉여금비율(%),총자산회전율(배),CASH FLOW 대 부채비율(%)
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [636]:
# 재무 공통 결측치 제거 
fin_index_drop = main_df.loc[main_df['당좌자산회전률(배)'].isna()].index
main_df.drop(index=fin_index_drop, inplace=True)

In [637]:
# 컬럼별 결측치 Check
for i in main_df.columns : 
    print(f'{i} : {main_df_copy_2[i].isna().sum()} -> {main_df[i].isna().sum()}')
    print('________________________')
    
main_df.shape

회사명 : 0 -> 0
________________________
거래소코드 : 0 -> 0
________________________
회계년도 : 0 -> 0
________________________
유동비율(%) : 96 -> 0
________________________
당좌비율(%) : 96 -> 0
________________________
비유동비율(%) : 96 -> 0
________________________
부채비율(%) : 96 -> 0
________________________
유동부채비율(%) : 96 -> 0
________________________
차입금의존도(%) : 96 -> 0
________________________
이익잉여금비율(%) : 192 -> 97
________________________
순운전자본비율(%) : 96 -> 0
________________________
비유동장기적합률(%) : 96 -> 0
________________________
이자보상배율(이자비용)(배) : 96 -> 0
________________________
외화포지션(배) : 192 -> 97
________________________
총자본순이익률(%) : 96 -> 0
________________________
자기자본순이익률(%) : 96 -> 0
________________________
경영자본순이익률(%) : 96 -> 0
________________________
매출액순이익률(%) : 96 -> 0
________________________
총자본정상영업이익률(%) : 96 -> 0
________________________
자기자본정상영업이익률(%) : 96 -> 0
________________________
매출액정상영업이익률(%) : 96 -> 0
________________________
금융비용부담률(%) : 96 -> 0
________________________
총자

(11152, 60)

In [638]:
main_df_copy_3 = main_df.copy()

## II-2. 시장정보(주가변수) 공통 NaN 제거
    - PBR, PER, PCR, 52주 베타, EV/EBITA
    - 11,152 -> 11,084 (68rows)
    - 제거 68rows 중 'Target' =1 : 12rows

In [639]:
# 68 rows 로 시장정보 NaN 값 갖는 row 동일 
main_df.loc[(main_df['PER'].isna())&(main_df['PER'].isna())&(main_df['PER'].isna()) & (main_df['EV/EBITDA(배)'].isna())& (main_df['52주베타(배)'].isna())]

# '1' : 12rows
# main_df.loc[(main_df['PER'].isna())&(main_df['PER'].isna())&(main_df['PER'].isna()) & (main_df['EV/EBITDA(배)'].isna())& (main_df['52주베타(배)'].isna())]['Target'].value_counts()

,회사명,거래소코드,회계년도,유동비율(%),당좌비율(%),비유동비율(%),부채비율(%),유동부채비율(%),차입금의존도(%),이익잉여금비율(%),...,최대주주변경횟수(회),감사법인변경횟수(회),업력(년),평균근속연수(년),직원수증감율(%),기간제비율(%),R&D집중도(%),수출기업여부(dummy),주거래은행,Target
6769,(주)영진코퍼레이션,053330,2014,39.68,39.68,0.00,0.00,0.00,42.83,-294.006991,...,2,1,13,NaN,-85.29,0.0,0.00,0,IBK기업은행,1
8631,(주)자비스,254120,2018,125.05,59.17,420.36,994.43,539.06,58.86,NaN,...,0,1,1,NaN,NaN,NaN,20.91,0,국민은행,0
9480,(주)카이노스메드,284620,2019,891.26,891.26,28.58,10.16,9.15,3.77,NaN,...,0,0,1,2.0,0.00,0.0,NaN,0,KEB하나은행,0
12003,(주)한국비엔씨,256840,2018,137.43,118.79,98.86,159.16,116.64,34.91,NaN,...,0,0,1,NaN,NaN,NaN,6.80,0,NH농협,0
12553,3노드디지탈그룹유한공사,900010,2012,112.82,94.91,104.01,232.47,202.50,35.54,21.267091,...,0,0,5,NaN,NaN,NaN,0.00,0,우리은행,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17342,헝셩그룹유한회사,900270,2018,411.65,402.21,25.18,24.89,24.22,12.81,69.810078,...,3,0,2,NaN,NaN,NaN,0.00,0,신한은행,0
17343,헝셩그룹유한회사,900270,2019,484.05,475.05,20.52,21.14,20.79,10.93,72.733622,...,4,0,3,NaN,NaN,NaN,0.00,0,신한은행,0
17344,헝셩그룹유한회사,900270,2020,515.18,506.99,23.44,18.53,18.46,10.56,74.670501,...,4,0,4,NaN,NaN,NaN,0.00,0,신한은행,0
17345,헝셩그룹유한회사,900270,2021,528.81,520.13,20.88,19.18,18.59,11.33,75.262694,...,4,0,5,NaN,NaN,NaN,0.00,0,신한은행,0


In [640]:
# 시장정보(주가변수) 공통 결측치 제거 
price_index_drop = main_df.loc[main_df['PER'].isna()].index
main_df.drop(index=price_index_drop, inplace=True)

In [641]:
# 컬럼별 결측치 Check
for i in main_df.columns : 
    print(f'{i} : {main_df_copy_3[i].isna().sum()} -> {main_df[i].isna().sum()}')
    print('________________________')

회사명 : 0 -> 0
________________________
거래소코드 : 0 -> 0
________________________
회계년도 : 0 -> 0
________________________
유동비율(%) : 0 -> 0
________________________
당좌비율(%) : 0 -> 0
________________________
비유동비율(%) : 0 -> 0
________________________
부채비율(%) : 0 -> 0
________________________
유동부채비율(%) : 0 -> 0
________________________
차입금의존도(%) : 0 -> 0
________________________
이익잉여금비율(%) : 97 -> 93
________________________
순운전자본비율(%) : 0 -> 0
________________________
비유동장기적합률(%) : 0 -> 0
________________________
이자보상배율(이자비용)(배) : 0 -> 0
________________________
외화포지션(배) : 97 -> 93
________________________
총자본순이익률(%) : 0 -> 0
________________________
자기자본순이익률(%) : 0 -> 0
________________________
경영자본순이익률(%) : 0 -> 0
________________________
매출액순이익률(%) : 0 -> 0
________________________
총자본정상영업이익률(%) : 0 -> 0
________________________
자기자본정상영업이익률(%) : 0 -> 0
________________________
매출액정상영업이익률(%) : 0 -> 0
________________________
금융비용부담률(%) : 0 -> 0
________________________
총자산회전율(배) : 97 -> 93


---

## II-3. NaN값 : 0을 의미하는 컬럼 대체 
    - R&D집중도
    - 기간제비율


In [642]:
main_df.loc[main_df['R&D집중도(%)'].isna(), 'R&D집중도(%)'] = 0

In [643]:
main_df.loc[main_df['기간제비율(%)'].isna(), '기간제비율(%)'] = 0 

In [644]:
main_df_copy_4 = main_df.copy()

## II-4. K2_score 기준 NaN 제거 : 95rows
    - 11,084 -> 10,989
    - Target 1 : 1 row

In [645]:
# K2_score기준 NaN 값 제거 : 95 rows 

print(main_df_copy_4['Target'].value_counts() ) # k2 NaN 제거 전 Target

main_df = main_df.loc[~(main_df['K2_score'].isna())]

main_df['Target'].value_counts()  # k2 NaN 제거 후 Target

0    10149
1      935
Name: Target, dtype: int64


0    10055
1      934
Name: Target, dtype: int64

In [646]:
# 컬럼별 결측치 Check
for i in main_df.columns : 
    print(f'{i} : {main_df_copy_4[i].isna().sum()} -> {main_df[i].isna().sum()}')
    print('________________________')

회사명 : 0 -> 0
________________________
거래소코드 : 0 -> 0
________________________
회계년도 : 0 -> 0
________________________
유동비율(%) : 0 -> 0
________________________
당좌비율(%) : 0 -> 0
________________________
비유동비율(%) : 0 -> 0
________________________
부채비율(%) : 0 -> 0
________________________
유동부채비율(%) : 0 -> 0
________________________
차입금의존도(%) : 0 -> 0
________________________
이익잉여금비율(%) : 93 -> 0
________________________
순운전자본비율(%) : 0 -> 0
________________________
비유동장기적합률(%) : 0 -> 0
________________________
이자보상배율(이자비용)(배) : 0 -> 0
________________________
외화포지션(배) : 93 -> 0
________________________
총자본순이익률(%) : 0 -> 0
________________________
자기자본순이익률(%) : 0 -> 0
________________________
경영자본순이익률(%) : 0 -> 0
________________________
매출액순이익률(%) : 0 -> 0
________________________
총자본정상영업이익률(%) : 0 -> 0
________________________
자기자본정상영업이익률(%) : 0 -> 0
________________________
매출액정상영업이익률(%) : 0 -> 0
________________________
금융비용부담률(%) : 0 -> 0
________________________
총자산회전율(배) : 93 -> 0
___

In [647]:
main_df_copy_5 = main_df.copy()
# main_df = main_df_copy_5.copy()

## II-2. 평균근속연수 Nan 제거 : 83 rows
    - 10,989 -> 10,906 
    - Target 1 : 17
    - 만일, 추후에 Feature Select 안될시, col 자체를 날리고 재분석 실행 예정

In [648]:
# 제거
print(main_df_copy_5[['Target']].value_counts()) 
main_df = main_df.loc[~(main_df['평균근속연수(년)'].isna())]
print(main_df[['Target']].value_counts()) 

Target
0         10055
1           934
dtype: int64
Target
0         9989
1          917
dtype: int64


In [649]:
# 컬럼별 결측치 Check
for i in main_df.columns : 
    print(f'{i} : {main_df_copy_5[i].isna().sum()} -> {main_df[i].isna().sum()}')
    print('________________________')

회사명 : 0 -> 0
________________________
거래소코드 : 0 -> 0
________________________
회계년도 : 0 -> 0
________________________
유동비율(%) : 0 -> 0
________________________
당좌비율(%) : 0 -> 0
________________________
비유동비율(%) : 0 -> 0
________________________
부채비율(%) : 0 -> 0
________________________
유동부채비율(%) : 0 -> 0
________________________
차입금의존도(%) : 0 -> 0
________________________
이익잉여금비율(%) : 0 -> 0
________________________
순운전자본비율(%) : 0 -> 0
________________________
비유동장기적합률(%) : 0 -> 0
________________________
이자보상배율(이자비용)(배) : 0 -> 0
________________________
외화포지션(배) : 0 -> 0
________________________
총자본순이익률(%) : 0 -> 0
________________________
자기자본순이익률(%) : 0 -> 0
________________________
경영자본순이익률(%) : 0 -> 0
________________________
매출액순이익률(%) : 0 -> 0
________________________
총자본정상영업이익률(%) : 0 -> 0
________________________
자기자본정상영업이익률(%) : 0 -> 0
________________________
매출액정상영업이익률(%) : 0 -> 0
________________________
금융비용부담률(%) : 0 -> 0
________________________
총자산회전율(배) : 0 -> 0
______

In [650]:
main_df_copy_6 = main_df.copy()

## II-3. 직원수증감율 Nan 제거 : 6 rows
    - 10,906 -> 10,900 
    - Target 1 : 1 
    -  파악불가
> 포인트엔지니어링 256630  
> 한국비엔씨 256840  
> 화신정공 126640  
> 휴럼 353190  
> 잘만테크 090120  
> 줌인터넷 239340  

In [651]:
main_df = main_df.loc[~(main_df['직원수증감율(%)'].isna())]

In [652]:
# 컬럼별 결측치 Check
for i in main_df.columns : 
    print(f'{i} : {main_df_copy_6[i].isna().sum()} -> {main_df[i].isna().sum()}')
    print('________________________')

회사명 : 0 -> 0
________________________
거래소코드 : 0 -> 0
________________________
회계년도 : 0 -> 0
________________________
유동비율(%) : 0 -> 0
________________________
당좌비율(%) : 0 -> 0
________________________
비유동비율(%) : 0 -> 0
________________________
부채비율(%) : 0 -> 0
________________________
유동부채비율(%) : 0 -> 0
________________________
차입금의존도(%) : 0 -> 0
________________________
이익잉여금비율(%) : 0 -> 0
________________________
순운전자본비율(%) : 0 -> 0
________________________
비유동장기적합률(%) : 0 -> 0
________________________
이자보상배율(이자비용)(배) : 0 -> 0
________________________
외화포지션(배) : 0 -> 0
________________________
총자본순이익률(%) : 0 -> 0
________________________
자기자본순이익률(%) : 0 -> 0
________________________
경영자본순이익률(%) : 0 -> 0
________________________
매출액순이익률(%) : 0 -> 0
________________________
총자본정상영업이익률(%) : 0 -> 0
________________________
자기자본정상영업이익률(%) : 0 -> 0
________________________
매출액정상영업이익률(%) : 0 -> 0
________________________
금융비용부담률(%) : 0 -> 0
________________________
총자산회전율(배) : 0 -> 0
______

In [653]:
print(main_df.shape)
main_df['Target'].value_counts()

(10900, 60)


0    9984
1     916
Name: Target, dtype: int64

In [654]:
main_df.to_excel('../dataset_real_final/main_df_dropna_completed.xlsx')

In [611]:
# 지우용 
# main_df.loc[main_df['회계년도'] >= 2018].to_excel('../dataset_real_final/main_df_2018~2022.xlsx')

# # 표준산업분류 drop
# main_df.drop('표준산업분류_중분류', axis=1, inplace=True)

---

In [612]:
main_df.loc[main_df["K2_score"]==-(np.inf)][["회사명","거래소코드","회계년도"]]

,회사명,거래소코드,회계년도
